In [4]:
!pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [294]:
!pip install geopy

     |████████████████████████████████| 118 kB 179 kB/s eta 0:00:01


In [162]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import numpy as np
from datetime import timedelta
from haversine import haversine
from random import randrange
from datetime import timedelta
from datetime import datetime

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import datetime as dt

from datetime import date
import holidays

# Загрузка данных

In [2]:
def parse_data(temp):

    res = pd.DataFrame()
    
    res['date'] = [temp.iloc[0][0]]
    res['number'] = temp.iloc[0][3]
    res['age'] = temp.iloc[0][16]
    res['who'] = temp.iloc[0][19]
    
    if temp.shape[0] == 4:

        res['occasion'] = temp.iloc[1][1]
        res['calling'] = temp.iloc[1][11]
        res['type'] = temp.iloc[1][18]
        
        res['delivered'] = temp.iloc[2][1]
        res['substation'] = temp.iloc[2][17]
        
        res['accept_time'] = temp.iloc[3][8]
        res['arrival_time'] = temp.iloc[3][11]
        res['hospitalized'] = temp.iloc[3][14]
    
    elif temp.shape[0] == 5:

        res['occasion'] = temp.iloc[1][1]
        res['calling'] = temp.iloc[1][11]
        res['type'] = temp.iloc[1][18]
        
        res['diagnosis'] = temp.iloc[2][1]
        res['result'] = temp.iloc[2][11]
        
        res['delivered'] = temp.iloc[3][1]
        res['substation'] = temp.iloc[3][17]
        
        res['accept_time'] = temp.iloc[4][8]
        res['arrival_time'] = temp.iloc[4][11]
        res['hospitalized'] = temp.iloc[4][14]
    
    else:
    
        res['adress'] = temp.iloc[1][1]
        
        res['occasion'] = temp.iloc[2][1]
        res['calling'] = temp.iloc[2][11]
        res['type'] = temp.iloc[2][18]
        
        res['diagnosis'] = temp.iloc[3][1]
        res['result'] = temp.iloc[3][11]
        
        res['delivered'] = temp.iloc[4][1]
        res['substation'] = temp.iloc[4][17]
        
        res['accept_time'] = temp.iloc[5][8]
        res['arrival_time'] = temp.iloc[5][11]
        res['hospitalized'] = temp.iloc[5][14]
    
    return res

In [3]:
def process_data(files):
    
    data_all = pd.DataFrame()
    
    for file in files:
        
        data_raw = pd.read_excel(file, header=None)
    
        i_start = []
        
        for i, val in enumerate(data_raw[0]):
            if type(data_raw[0][i]) == type(data_raw[0][0]) and type(data_raw[3][i]) == str and i > 0:
                i_start.append(i)
                
        data = pd.DataFrame()
        for j in tqdm(range(len(i_start)-1)):
            temp = parse_data(data_raw.iloc[i_start[j]:i_start[j+1]])
            data = pd.concat([data, temp], ignore_index=True)
            
        data_all = pd.concat([data_all, data], ignore_index=True)
        
    return data_all

In [361]:
data = process_data(glob('data/Журнал активных вызовов 2022/*'))

100%|████████████████████████████████████| 10887/10887 [01:28<00:00, 122.49it/s]


In [1098]:
# data.to_csv('data_2022.csv', index=False)

data = pd.read_csv('data_2022.csv')

In [1101]:
data = data[(data.accept_time.notnull())&(data.arrival_time.notnull())&(data.substation.notnull())]

In [1102]:
data = data.drop(['hospitalized'], axis=1)

In [1103]:
data.date = pd.to_datetime(data.date)

In [1104]:
data['accept_datetime'] = data[['date', 'accept_time']].apply(lambda x: pd.to_datetime(str(x[0].date()) + ' ' + str(x[1])), axis=1)

# Добавление нулевых вызовов

In [1106]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

d1 = datetime.strptime('1/1/2022 1:00 AM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('5/24/2022 11:50 PM', '%m/%d/%Y %I:%M %p')

random_datetimes = []

for i in range(data.shape[0]):
    random_datetimes.append(random_date(d1, d2))

In [1107]:
random_data = pd.DataFrame()
random_data['accept_datetime'] = random_datetimes
random_data['substation'] = data.substation.values
random_data['who'] = ''
random_data['adress'] = ''
random_data['occasion'] = ''
random_data['calling'] = ''
random_data['type'] = ''
random_data['diagnosis'] = ''
random_data['result'] = ''

In [1108]:
data = data.sort_values('accept_datetime').reset_index(drop=True)
random_data = random_data.sort_values('accept_datetime').reset_index(drop=True)

## Подстанции

In [37]:
substations = pd.read_excel('substations.xlsx')
# pd.DataFrame(data.substation.unique()).to_excel('substations.xlsx', index=False)
substations['latitude'] = substations.coords.apply(lambda x: float(x.split(',')[0]))
substations['longitude'] = substations.coords.apply(lambda x: float(x.split(',')[1]))

In [1110]:
data = pd.merge(data, substations, how='left',
                left_on='substation', right_on='name')

random_data = pd.merge(random_data, substations, how='left',
                       left_on='substation', right_on='name')

## Возраст и время

In [1111]:
def process_age(x):
    if 'лет' in x:
        return float(x.split(' ')[0])
    elif 'месяцев' in x:
        return float(x.split(' ')[0]) / 12
    elif 'недель' in x:
        return float(x.split(' ')[0]) / 12 / 4
    
data['age'] = data.age.apply(lambda x: process_age(x))

In [1112]:
data = data.drop_duplicates('number').reset_index(drop=True)

In [1113]:
data.coords = data.coords.apply(lambda x: [float(x.split(', ')[0]), float(x.split(', ')[1])])

In [1114]:
data['road_time'] = data[['arrival_time', 'accept_time']].apply(lambda x: (pd.to_datetime(x[0]) - pd.to_datetime(x[1])).seconds / 60, axis=1)

In [1115]:
data.accept_time = data.accept_time.apply(lambda x: pd.to_datetime(x).time())
data.arrival_time = data.arrival_time.apply(lambda x: pd.to_datetime(x).time())

In [1116]:
data.date = pd.to_datetime(data.date)

In [1117]:
data['weekday'] = data.date.apply(lambda x: x.weekday())
data['day'] = data.date.apply(lambda x: x.day)
data['week'] = data.date.apply(lambda x: x.week)
data['month'] = data.date.apply(lambda x: x.month)

In [1118]:
random_data['weekday'] = random_data.accept_datetime.apply(lambda x: x.weekday())
random_data['day'] = random_data.accept_datetime.apply(lambda x: x.day)
random_data['week'] = random_data.accept_datetime.apply(lambda x: x.week)
random_data['month'] = random_data.accept_datetime.apply(lambda x: x.month)

In [ ]:
data['hour'] = data.accept_datetime.apply(lambda x: x.hour)
random_data['hour'] = random_data.accept_datetime.apply(lambda x: x.hour)

In [1120]:
data['holiday'] = data.date.apply(lambda x: holidays.RUS().get(x))
random_data['holiday'] = random_data.accept_datetime.apply(lambda x: holidays.RUS().get(x))

In [1121]:
data.holiday.value_counts()

Новый год                   2366
Православное Рождество       340
День защитника отечества     270
Праздник Весны и Труда       236
День женщин                  229
День Победы                  204
Name: holiday, dtype: int64

In [1122]:
random_data.holiday.value_counts()

Новый год                   1937
Православное Рождество       301
День защитника отечества     285
Праздник Весны и Труда       278
День Победы                  270
День женщин                  268
Name: holiday, dtype: int64

In [1123]:
data.to_csv('data_2022_processed.csv', index=False)
random_data.to_csv('random_data_2022_processed.csv', index=False)

In [4]:
data = pd.read_csv('data_2022_processed.csv')
random_data = pd.read_csv('random_data_2022_processed.csv')

# Исследование заболеваний

In [626]:
data.occasion

0                                                      NaN
1                         Повышение ад без признаков криза
2        Температура 38-39, ребёнок >1 года, других жал...
3                      Плохо (после употребления алкоголя)
4                                 Рвота, понос у взрослого
                               ...                        
37253                                Температура, взрослый
37254              Задыхается (нарушение сердечного ритма)
37255                     Повышение ад без признаков криза
37256                         Задыхается, кашель (ребенок)
37257                                    Констатация, умер
Name: occasion, Length: 37258, dtype: object

In [456]:
data.result.unique()

array(['оказана помощь, больной оставлен на месте',
       'отказ от транспортировки для госпитализации в стационар',
       'смерть до приезда бригады СМП', 'доставлен в больницу',
       'передан бригаде СПЦ', nan, 'смерть в присутствии бригады СМП',
       'отказ от помощи', 'больной не найден на месте',
       'немедицинская перевозка', 'доставлен в дом ночного прибывания',
       'обслужен врачом до прибытия СМП', 'пациент практически здоров',
       'больной увезен до прибытия СМП', 'вызов отменен',
       'доставлен в травмпункт', 'передан другой бригаде',
       'отказ от осмотра', 'скорую не вызывали'], dtype=object)

In [460]:
data.diagnosis.nunique()

710

In [464]:
print(data.calling.nunique())
print(data.calling.unique())

6
['Первичный' 'Повторный' 'Попутный' nan 'Неотложка' 'Амбулаторный'
 'Активный']


In [465]:
print(data.type.nunique())
print(data.type.unique())

6
['неотложное состояние' 'внезапное заболевание' 'несчастный случай' nan
 'хроническое заболевание' 'патология беременности' 'роды']


In [473]:
pd.DataFrame(data[data.occasion.notnull()].occasion.value_counts()).to_excel('occasions.xlsx')

In [474]:
pd.DataFrame(data[data.result.notnull()].result.value_counts()).to_excel('call_result.xlsx')

# Погода

In [5]:
weather = pd.read_csv('weather_2022.csv', encoding='utf-8', skiprows=6, sep=';', index_col=0)

weather_columns = weather.columns[1:]
weather = weather.drop(weather.columns[-1], axis=1)
weather.columns = weather_columns

weather = weather.reset_index().rename(columns={'index':'datetime'}).sort_values('datetime').reset_index(drop=True)

weather = weather[['datetime', 'T', 'Tg', 'Po', 'Pa', 'U', 'ff3']]

weather.datetime = pd.to_datetime(weather.datetime)

In [8]:
data.date = pd.to_datetime(data.date)

In [9]:
data['accept_datetime'] = data[['date', 'accept_time']].apply(lambda x: pd.to_datetime(str(x[0].date()) + ' ' + str(x[1])), axis=1)

In [10]:
weather_for_data = pd.DataFrame()

for adt in tqdm(data['accept_datetime']):
    weather_for_data = pd.concat([weather_for_data, weather[weather.datetime <= adt].iloc[-1:]],
                                 ignore_index=True)
    
data[weather_for_data.columns[1:]] = weather_for_data[weather_for_data.columns[1:]]

100%|████████████████████████████████████| 37258/37258 [00:38<00:00, 971.57it/s]


In [11]:
weather_for_data = pd.DataFrame()

for adt in tqdm(random_data['accept_datetime']):
    weather_for_data = pd.concat([weather_for_data, weather[weather.datetime <= adt].iloc[-1:]],
                                 ignore_index=True)
    
random_data[weather_for_data.columns[1:]] = weather_for_data[weather_for_data.columns[1:]]

100%|████████████████████████████████████| 40221/40221 [00:41<00:00, 961.30it/s]


# ДТП (проработали, но не использовали)

In [713]:
# dtp = pd.read_csv('dtp201804-1.csv', encoding='cp1251', sep=';')
# dtp = dtp[dtp.reg_name == 'Нижегородская область'].reset_index(drop=True)
# for i in range(len(substations)):
#     latlon = tuple(substations[['latitude', 'longitude']].iloc[i])
#     dtp[substations.name.iloc[i]] = dtp[['latitude', 'longitude']].apply(lambda x: haversine((x[0], x[1]), latlon), axis=1)

In [886]:
# dtp = pd.concat([pd.read_excel(file) for file in glob('дтп/*')])
# dtp['Место ДТП'] = dtp['Место ДТП'].apply(lambda x: ''.join(x.split(', ')[2:3]))
# dtp['datetime'] = pd.to_datetime(dtp['Дата-время ДТП'])
# dtp['victims'] = dtp['Погибло'] + dtp['Ранено']
# dtp.drop_duplicates('Номер ДТП').reset_index(drop=True)
# dtp.datetime = pd.to_datetime(dtp.datetime.apply(lambda x: str(x).replace('2021', '2022')))
# # pd.DataFrame(dtp['Место ДТП'].value_counts()).to_excel('dtp_regions.xlsx')

# dtp_regions = pd.read_excel('dtp_regions.xlsx')
# dtp_regions.coords = dtp_regions.coords.apply(lambda x: tuple([float(x.split(', ')[0]), float(x.split(', ')[1])]) )

In [734]:
# data.coords = data.coords.apply(lambda x: tuple(x))

In [776]:
# substations.coords = substations.coords.apply(lambda x: tuple([float(x.split(', ')[0]), float(x.split(', ')[1])]))

In [777]:
# substations = substations[substations.columns[:4]]

In [778]:
# dtp_subs = pd.DataFrame()

# for j in range(len(dtp_regions)):
    
#     dtp_reg = dtp_regions.region.iloc[j]
#     dtp_coords = dtp_regions.coords.iloc[j]
    
#     temp_hav = []

#     for i in range(len(substations)):
    
#         name = substations.name.iloc[i]
#         coords = substations.coords.iloc[i]
    
#         temp_hav.append(haversine(dtp_coords, coords))
        
#     dtp_subs[dtp_reg] = temp_hav

In [780]:
# dtp_subs = dtp_subs.T
# dtp_subs.columns = list(substations.name)

In [791]:
# # до 20 км
# list(dtp_subs[dtp_subs[dtp_subs.columns[5]] < 20].index)

['Нижний Новгород', 'Городской округ г. Бор']

In [792]:
# dtp_subs_dict = {}
# for col in dtp_subs.columns:
#     dtp_subs_dict[col] = list(dtp_subs[dtp_subs[col] < 20].index)

In [887]:
# data['dtp_datetime'] = data.accept_datetime - timedelta(hours=2)

In [893]:
# dtp_data = pd.DataFrame()

# for i in tqdm(range(data.shape[0])):

#     dttm = data.accept_datetime.iloc[i]
#     dttm_dtp = data.dtp_datetime.iloc[i]
#     dtp_reg = data.substation.iloc[i]
    
#     temp_dtp = dtp[(dtp['Место ДТП'].isin(dtp_subs_dict[dtp_reg]))
#                    &(pd.to_datetime(dtp.datetime) > dttm_dtp)&(pd.to_datetime(dtp.datetime) < dttm)].copy()
#     if temp_dtp.shape[0] > 0:
#         temp_dtp.loc[:, 'accept_datetime'] = dttm
#         temp_dtp.loc[:, 'substation'] = dtp_reg
#         dtp_data = pd.concat([dtp_data, temp_dtp.drop_duplicates('Номер ДТП')], axis=0)

100%|████████████████████████████████████| 37258/37258 [04:21<00:00, 142.36it/s]


# Dummy

In [895]:
data.columns

Index(['date', 'number', 'age', 'who', 'adress', 'occasion', 'calling', 'type',
       'diagnosis', 'result', 'delivered', 'substation', 'accept_time',
       'arrival_time', 'name', 'coords', 'latitude', 'longitude', 'road_time',
       'weekday', 'day', 'week', 'month', 'holiday', 'accept_datetime', 'T',
       'Tg', 'Po', 'Pa', 'U', 'ff3', 'dtp_datetime'],
      dtype='object')

In [ ]:
# TO DUMMY

# occasion
# result
# calling
# type

# substation

# holiday

In [482]:
temp = data[['latitude', 'longitude', 'road_time', 'weekday', 'day', 'week', 'month']]

In [486]:
for col in ['occasion', 'result', 'calling', 'type', 'substation', 'holiday']:
    temp = pd.concat([temp, pd.get_dummies(data[col])], axis=1)

In [581]:
temp.corrwith(temp.road_time).sort_values().head(10)

БСМП г. Дзержинск Юго-Западная          -0.021176
БСМП г. Дзержинск Центральная           -0.017843
ПСМП №6                                 -0.014405
Попутный                                -0.013730
ПСМП №4                                 -0.013507
восстановление по бумажной технологии   -0.012791
Чкаловская ЦРБ                          -0.010683
БСМП г. Арзамас                         -0.010593
Сергачская ЦРБ                          -0.010284
ПСМП №1                                 -0.009748
dtype: float64

In [582]:
temp.corrwith(temp.road_time).sort_values().tail(10)

ПСМП №2                                 0.011817
Первичный                               0.011820
Володарская ЦРБ Ильиногорская           0.012783
Кстовская ЦРБ                           0.013039
Арзамасская РБ                          0.017940
Авария (дтп) 1-2 пострадавших           0.020998
Субфебрильная температура у взрослых    0.022439
Борская ЦРБ                             0.029503
Спасская ЦРБ                            0.040700
road_time                               1.000000
dtype: float64

# Обучение модели

In [13]:
random_data['road_time'] = 0
for i, subst in tqdm(enumerate(random_data.substation.values)):
    random_data.loc[i, 'road_time'] = data.loc[random_data.substation == subst, 'road_time'].sample(1).iloc[0]

40221it [03:03, 218.89it/s]


In [14]:
random_data.road_time.isnull().sum()

0

In [92]:
random_data.accept_datetime = pd.to_datetime(random_data.accept_datetime)

In [94]:
data['hour'] = data.accept_datetime.apply(lambda x: x.hour)
random_data['hour'] = random_data.accept_datetime.apply(lambda x: x.hour)

## Предсказание вызова

In [153]:
data['call_target'] = 1
random_data['call_target'] = 0
may_data['call_target'] = 1

train_data = pd.concat([data, random_data])[[
    # 'road_time', 
                                             'hour', 'weekday', 'day', 'week', 'month', 'holiday', 
                                             'T', 'Tg', 'Po', 'Pa', 'U', 'ff3', 'substation', 'call_target']]



cat_features = ['substation', 'holiday']
train_data[cat_features] = train_data[cat_features].fillna('')

In [154]:
X_train, X_val, y_train, y_val = train_test_split(train_data.drop(['call_target'], axis=1),
                                                  train_data['call_target'],
                                                  stratify=train_data.substation,
                                                  # stratify=train_data['call_target'].astype(str) + '+' + train_data['substation'],
                                                  test_size=0.25)

In [155]:
model = CatBoostClassifier(verbose=100, cat_features=cat_features,
                           loss_function='MultiClassOneVsAll'
                          )
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	learn: 0.6920203	test: 0.6919483	best: 0.6919483 (0)	total: 29.9ms	remaining: 29.9s
100:	learn: 0.6678928	test: 0.6670317	best: 0.6670317 (100)	total: 3.61s	remaining: 32.2s
200:	learn: 0.6645517	test: 0.6644874	best: 0.6644874 (200)	total: 7.26s	remaining: 28.9s
300:	learn: 0.6622460	test: 0.6631205	best: 0.6631205 (300)	total: 10.6s	remaining: 24.7s
400:	learn: 0.6604730	test: 0.6622451	best: 0.6622451 (400)	total: 13.9s	remaining: 20.8s
500:	learn: 0.6583648	test: 0.6612667	best: 0.6612667 (500)	total: 17.4s	remaining: 17.3s
600:	learn: 0.6566661	test: 0.6607221	best: 0.6607221 (600)	total: 20.8s	remaining: 13.8s
700:	learn: 0.6552202	test: 0.6604109	best: 0.6604109 (700)	total: 24.3s	remaining: 10.4s
800:	learn: 0.6539773	test: 0.6601247	best: 0.6601247 (800)	total: 27.6s	remaining: 6.86s
900:	learn: 0.6528319	test: 0.6598577	best: 0.6598563 (897)	total: 30.9s	remaining: 3.4s
999:	learn: 0.6518270	test: 0.6597688	best: 0.6597688 (999)	total: 33.9s	remaining: 0us

bestTest = 0.65

In [156]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

           0       0.61      0.65      0.63     10036
           1       0.59      0.55      0.57      9334

    accuracy                           0.60     19370
   macro avg       0.60      0.60      0.60     19370
weighted avg       0.60      0.60      0.60     19370



In [157]:
fip = pd.DataFrame()
fip['feature'] = model.feature_names_
fip['importance'] = model.get_feature_importance()
fip = fip.sort_values('importance', ascending=False)

In [158]:
fip

,feature,importance
0,hour,34.585067
12,substation,17.205466
3,week,11.467148
2,day,10.536961
4,month,5.381643
1,weekday,4.941126
5,holiday,3.226552
10,U,2.950164
8,Po,2.920941
6,T,2.659290


## Инференс предсказаний вызова

In [28]:
data.loc[data.substation == 'ПСМП №5', ['adress', 'substation', 'road_time']].road_time.describe()

count    2670.000000
mean       14.239813
std        39.792961
min         0.000000
25%         8.583333
50%        12.850000
75%        16.433333
max      1439.850000
Name: road_time, dtype: float64

In [29]:
test = data.groupby('substation').road_time.mean().reset_index().copy()
test['datetime'] = dt.datetime.now()

In [32]:
test['weekday'] = test.datetime.apply(lambda x: x.weekday())
test['day'] = test.datetime.apply(lambda x: x.day)
test['week'] = test.datetime.apply(lambda x: x.week)
test['month'] = test.datetime.apply(lambda x: x.month)
test['holiday'] = test.datetime.apply(lambda x: holidays.RUS().get(x))
test['holiday'] = test['holiday'].fillna('')

In [33]:
weather_for_data = pd.DataFrame()

for adt in tqdm(test['datetime']):
    weather_for_data = pd.concat([weather_for_data, weather[weather.datetime <= adt].iloc[-1:]],
                                 ignore_index=True)
    
test[weather_for_data.columns[1:]] = weather_for_data[weather_for_data.columns[1:]]

100%|██████████████████████████████████████████| 46/46 [00:00<00:00, 825.16it/s]


In [34]:
test[model.feature_names_]

,road_time,weekday,day,week,month,holiday,T,Tg,Po,Pa,U,ff3,substation
0,5.800000,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Ардатовская ЦРБ
1,18.556600,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Арзамасская РБ
2,10.691313,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,БСМП г. Арзамас
3,9.362574,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,БСМП г. Дзержинск Центральная
4,9.552639,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,БСМП г. Дзержинск Юго-Западная
5,22.328623,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Богородская ЦРБ
6,7.000000,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Большемурашкинская ЦРБ
7,19.045431,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Борская ЦРБ
8,18.959343,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Борская ЦРБ СМП Затон ППК
9,18.788889,6,5,22,6,,17.7,NaN,752.4,0.3,54,NaN,Борская ЦРБ СМП Кантаурово


In [35]:
model.predict_proba(test[model.feature_names_])[:, -1]

array([0.45535026, 0.3729827 , 0.35549121, 0.38619559, 0.40865863,
       0.4185179 , 0.32492091, 0.52621074, 0.19142581, 0.28281644,
       0.15313981, 0.30128048, 0.55102889, 0.27064822, 0.54231124,
       0.28866809, 0.54084907, 0.1419914 , 0.28720958, 0.1624443 ,
       0.3656728 , 0.44982808, 0.23666726, 0.27609169, 0.49937537,
       0.47969218, 0.55086509, 0.37980376, 0.50782634, 0.32199877,
       0.56907455, 0.51338328, 0.48671107, 0.22270804, 0.23804965,
       0.23007421, 0.16038996, 0.19620073, 0.41441824, 0.26912106,
       0.4837916 , 0.56070722, 0.26602662, 0.18441056, 0.37152167,
       0.14193726])

In [38]:
test_res = pd.DataFrame()
test_res['substation'] = test.substation
# test_res = 
test_res = pd.merge(test_res, substations[['name', 'latitude', 'longitude']], 
                    left_on='substation', right_on='name', how='left')
test_res['call_target'] = model.predict_proba(test[model.feature_names_])[:, -1]

In [39]:
test_res.to_csv('test_res.csv', index=False)

In [40]:
test_res

,substation,name,latitude,longitude,call_target
0,Ардатовская ЦРБ,Ардатовская ЦРБ,55.236419,43.110220,0.455350
1,Арзамасская РБ,Арзамасская РБ,55.386209,43.776236,0.372983
2,БСМП г. Арзамас,БСМП г. Арзамас,55.398313,43.820556,0.355491
3,БСМП г. Дзержинск Центральная,БСМП г. Дзержинск Центральная,56.245820,43.473968,0.386196
4,БСМП г. Дзержинск Юго-Западная,БСМП г. Дзержинск Юго-Западная,56.223359,43.381932,0.408659
5,Богородская ЦРБ,Богородская ЦРБ,56.098540,43.532799,0.418518
6,Большемурашкинская ЦРБ,Большемурашкинская ЦРБ,55.785696,44.786794,0.324921
7,Борская ЦРБ,Борская ЦРБ,56.378690,44.026078,0.526211
8,Борская ЦРБ СМП Затон ППК,Борская ЦРБ СМП Затон ППК,56.358701,44.061529,0.191426
9,Борская ЦРБ СМП Кантаурово,Борская ЦРБ СМП Кантаурово,56.486385,44.039044,0.282816


In [41]:
# substation
# данные по времени
# погода (через яндекс апи, брать только НН, каждую минуту)
# road_time - накинуть эвристику

In [42]:
substations.name

0                     Арзамасская РБ
1             Большемурашкинская ЦРБ
2                            ПСМП №1
3                            ПСМП №7
4                            ПСМП №2
5                            ПСМП №9
6                            ПСМП №8
7                            ПСМП №4
8                        Борская ЦРБ
9                            ПСМП №5
10                   БСМП г. Арзамас
11                     Кстовская ЦРБ
12                           ПСМП №6
13                    Павловская ЦРБ
14     БСМП г. Дзержинск Центральная
15    БСМП г. Дзержинск Юго-Западная
16                    Выксунская ЦРБ
17                           ПСМП №3
18                   Воротынская ЦРБ
19                       Вадская ЦРБ
20                    Лысковская ЦРБ
21                    Сергачская ЦРБ
22                   Кулебакская ЦРБ
23                   Володарская ЦРБ
24                   Пильнинская ЦРБ
25                  Первомайская ЦРБ
26                   Семеновская ЦРБ
2

In [1183]:
import datetime as dt

# Save the current time to a variable ('t')
t = dt.datetime.now()

while True:
    delta = dt.datetime.now()-t
    if delta.seconds >= 60:
        print("1 Min")
        # Update 't' variable to new time
        t = dt.datetime.now()

1 Min


KeyboardInterrupt: 

## Предсказание тяжести

In [1155]:
first_occasions = pd.read_csv('Первичные признаки - Лист1.csv')
first_occasions = first_occasions.rename(columns={'Тяжесть (0 - норма, 1 - тяжелое)':'heaviness',
                                                  'Признаки':'occasion'})
first_occasions = first_occasions[['occasion', 'heaviness']]
first_occasions = first_occasions[first_occasions.heaviness.notnull()]

In [1159]:
data = pd.merge(data, first_occasions, how='left', on='occasion')

In [1176]:
random_data['heaviness'] = 0

train_data = pd.concat([data[data.heaviness.notnull()], random_data])[['road_time', 
                                             'weekday', 'day', 'week', 'month', 'holiday', 
                                             'T', 'Tg', 'Po', 'Pa', 'U', 'ff3', 'substation', 'heaviness']]

cat_features = ['substation', 'holiday']
train_data[cat_features] = train_data[cat_features].fillna('')
train_data.heaviness = train_data.heaviness.astype(int)

In [1177]:
(train_data['heaviness'].astype(str) + '+' + train_data['substation']).value_counts()

0+ПСМП №3                  4425
0+ПСМП №7                  4030
0+ПСМП №1                  3741
0+ПСМП №5                  3667
0+ПСМП №9                  3256
                           ... 
0+Ардатовская ЦРБ             2
1+Тонкинская ЦРБ              1
1+Борская ЦРБ СМП Линда       1
1+Ардатовская ЦРБ             1
0+Тонкинская ЦРБ              1
Length: 88, dtype: int64

In [1178]:
X_train, X_val, y_train, y_val = train_test_split(train_data.drop(['heaviness'], axis=1),
                                                  train_data['heaviness'],
                                                  stratify=train_data.substation,
                                                  # stratify=train_data['call_target'].astype(str) + '+' + train_data['substation'],
                                                  test_size=0.25)

In [1179]:
model = CatBoostClassifier(verbose=100, cat_features=cat_features)
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.085274
0:	learn: 0.6740234	test: 0.6739612	best: 0.6739612 (0)	total: 19.9ms	remaining: 19.9s
100:	learn: 0.5546803	test: 0.5564104	best: 0.5563937 (97)	total: 2.24s	remaining: 19.9s
200:	learn: 0.5457281	test: 0.5527205	best: 0.5527193 (198)	total: 4.45s	remaining: 17.7s
300:	learn: 0.5396251	test: 0.5521404	best: 0.5521125 (297)	total: 6.71s	remaining: 15.6s
400:	learn: 0.5349717	test: 0.5519363	best: 0.5519095 (391)	total: 8.95s	remaining: 13.4s
500:	learn: 0.5305217	test: 0.5519196	best: 0.5517731 (432)	total: 11.3s	remaining: 11.2s
600:	learn: 0.5267281	test: 0.5520322	best: 0.5517731 (432)	total: 13.5s	remaining: 8.98s
700:	learn: 0.5229613	test: 0.5522024	best: 0.5517731 (432)	total: 15.9s	remaining: 6.78s
800:	learn: 0.5193957	test: 0.5525548	best: 0.5517731 (432)	total: 18.5s	remaining: 4.6s
900:	learn: 0.5159913	test: 0.5528804	best: 0.5517731 (432)	total: 21.2s	remaining: 2.33s
999:	learn: 0.5129854	test: 0.5531706	best: 0.5517731 (432)	total: 23.8s	re

In [1180]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

           0       0.73      0.99      0.85     13452
           1       0.44      0.01      0.03      4889

    accuracy                           0.73     18341
   macro avg       0.59      0.50      0.44     18341
weighted avg       0.66      0.73      0.63     18341



In [1181]:
fip = pd.DataFrame()
fip['feature'] = model.feature_names_
fip['importance'] = model.get_feature_importance()
fip = fip.sort_values('importance', ascending=False)

In [1182]:
X_train.head(1)

,road_time,weekday,day,week,month,holiday,T,Tg,Po,Pa,U,ff3,substation
35855,18.983333,1,17,20,5,,-0.2,NaN,745.3,0.0,45,4.0,ПСМП №1


In [1141]:
fip

,feature,importance
0,road_time,32.202826
12,substation,25.126436
2,day,7.244547
3,week,5.226314
9,Pa,4.852824
6,T,4.766593
8,Po,4.647524
1,weekday,3.744675
10,U,3.326814
5,holiday,3.269516


# Координаты по адресам

In [127]:
nn_substations = ['ПСМП №5', 'ПСМП №7', 'ПСМП №1', 'ПСМП №3', 'ПСМП №8', 'ПСМП №4', 'ПСМП №2', 'ПСМП №9', 'ПСМП №6']

In [128]:
may_data = data[(data.date >= pd.to_datetime('2022-05-01'))].copy()

In [129]:
nn_addresses = may_data[may_data.substation.isin(nn_substations)
                        &(may_data.adress.notnull())].adress.apply(lambda x: ', '.join(x.split(', ')[:3]))

In [130]:
may_data.loc[may_data.substation.isin(nn_substations)&(may_data.adress.notnull()), 'nn_address'] = nn_addresses

In [131]:
nn_addresses = nn_addresses.unique()

In [132]:
len(nn_addresses)

1083

In [133]:
temp_address = nn_addresses[1]
print(temp_address)

г.Нижний Новгород, ул. 40 лет Победы, д.7


In [134]:
# from geopy.geocoders import Yandex
# import time
# geolocator = Yandex('74dc0c13-a2df-42dc-b5ca-c4d3d306ae4d')

# locations = []

# for temp_address in tqdm(nn_addresses):
#     location = geolocator.geocode(temp_address)
#     if len(location) > 0 and len(location[1]) == 2:
#         locations.append((location.latitude, location.longitude))
#     else:
#         locations.append((0, 0))

In [135]:
# locs = pd.DataFrame()
# locs['nn_address'] = nn_addresses
# locs['nn_coords'] = locations
# locs['latitude'] = locs.nn_coords.apply(lambda x: x[0])
# locs['longitude'] = locs.nn_coords.apply(lambda x: x[1])

In [136]:
# locs = locs[(locs.latitude > 50)&(locs.latitude < 60)
#             &(locs.longitude > 39)&(locs.longitude < 50)].reset_index(drop=True)

In [137]:
# locs.to_csv('nn_coords', index=False)

In [138]:
locs = pd.read_csv('nn_coords')

In [139]:
may_data = pd.merge(may_data, locs, on='nn_address', how='inner')

In [144]:
# may_data.coords = may_data.coords.apply(lambda x: tuple([float(x.split(',' )[0][1:]), float(x.split(',' )[1][:-1])]))
# may_data.nn_coords = may_data.nn_coords.apply(lambda x: tuple([float(x.split(',' )[0][1:]), float(x.split(',' )[1][:-1])]))

In [146]:
may_data['dist'] = may_data[['coords', 'nn_coords']].apply(lambda x: haversine(x[0], x[1]), axis=1)

In [98]:
may_data.to_csv('may_data.csv', index=False)